In [81]:
import json
import re
from pathlib import Path
from typing import List, Dict, Union, Optional
from PIL import Image
import pytesseract
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

# 设置 Tesseract 路径（Windows 用户）
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# ===========================================================
#  工具函数：读取配置文件
# ===========================================================
def load_config(config_path: str) -> Dict:
    """加载配置文件"""
    with open(config_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# ===========================================================
#  工具函数：裁剪表格区域
# ===========================================================
def crop_table_area(img: Image.Image, config: Dict) -> Image.Image:
    """根据配置文件中的比例裁剪表格区域"""
    bounds = config["table_area"]["bounds"]
    w, h = img.size
    left = int(w * bounds["left_ratio"])
    right = int(w * bounds["right_ratio"])
    top = int(h * bounds["top_ratio"])
    bottom = int(h * bounds["bottom_ratio"])
    return img.crop((left, top, right, bottom))

# 明日方舟图片预处理
def improve_for_arknights(img: Image.Image) -> Image.Image:
    """
    对图片进行特定预处理以提高 OCR 准确率
    """    
    im = img.copy()
    img_array = np.array(im)
    #　将特定颜色的像素（RGB(31,31,31)以及一定范围内相似的像素）设为黑色。
    target_color = np.array([31, 31, 31])
    tolerance = 15
    
    # 计算颜色差异
    color_diff = np.abs(img_array[:, :, :3] - target_color)
    mask = np.all(color_diff <= tolerance, axis=2)
    
    # 统一处理方式：无论图像模式如何，都只设置RGB通道为黑色
    # 对于有alpha通道的图像，alpha通道保持不变
    img_array[mask, :3] = [0, 0, 0]
    # 返回处理后的 PIL.Image
    processed_img = Image.fromarray(img_array)
    return processed_img

# ===========================================================
#  工具函数：OCR 识别
# ===========================================================
def try_ocr(img: Image.Image, lang: str = "chi_sim") -> Optional[str]:
    """使用 pytesseract 对图片执行 OCR 并返回文本"""
    try:
        text = pytesseract.image_to_string(img, lang=lang)
        print(f"text: \n{text}")
        return text.strip() if text else None
    except Exception as e:
        print(f"[try_ocr] OCR 失败: {e}")
        return None


In [82]:
# ===========================================================
#  工具函数：解析 OCR 文本
# ===========================================================
def parse_ocr_text(ocr_text: str, config: Dict) -> List[Dict]:
    """将 OCR 文本解析为结构化条目"""
    if not ocr_text:
        return []

    column_indices = config["table_area"]["column_indices"]
    lines = [line.strip() for line in ocr_text.splitlines() if line.strip()]
    entries = []

    # 定义支持的日期格式正则表达式
    date_patterns = [
    r'(\d{4}[-/]\d{1,2}[-/]\d{1,2}[ T]\d{1,2}:\d{2}:\d{2})',  # 日期时间（带秒）
    r'(\d{4}[-/]\d{1,2}[-/]\d{1,2}[ T]\d{1,2}:\d{2})',        # 日期时间（不带秒）
    r'(\d{4}[-/]\d{1,2}[-/]\d{1,2})',                         # 日期（不带时间）
    ]

    for line in lines:
        ts_match = None
        for pattern in date_patterns:
            ts_match = re.search(pattern, line)
            if ts_match:
                break  # 找到匹配的日期格式后退出循环

        if ts_match:
            ts = ts_match.group(1)
            left_part = line[:ts_match.start()].strip(" |,-\t")

            # 优先尝试用竖线分割，如果没有竖线，再按多个连续空格分割
            if "|" in left_part:
                parts = [p.strip() for p in left_part.split("|")]
            else:
                parts = re.split(r"\s{2,}", left_part)

            item = parts[column_indices["item"]].strip() if len(parts) > column_indices["item"] else ""
            pool = parts[column_indices["pool"]].strip() if len(parts) > column_indices["pool"] else ""

            entries.append({"item": item, "pool": pool, "time": ts})
    print(f"entries: {entries}")
    return entries


In [83]:
# ===========================================================
#  工具函数：修复时间
# ===========================================================
def fix_timestamp(ts: str) -> Optional[str]:
    """修复时间格式"""
    if not ts:
        return None

    s = ts.strip().replace("T", " ").replace(".", "-").replace("/", "-")
    s = re.sub(r'[年月日]', '-', s)
    s = re.sub(r'[^0-9\- :]', '', s)

    candidates = [
        "%Y-%m-%d %H:%M:%S", "%Y-%m-%d %H:%M", "%Y-%m-%d",
        "%Y%m%d %H%M%S", "%Y%m%d%H%M%S"
    ]

    for fmt in candidates:
        try:
            dt = datetime.strptime(s, fmt)
            return dt.strftime("%Y-%m-%d %H:%M:%S")
        except:
            continue

    digits = re.sub(r'\D', '', s)
    if len(digits) >= 14:
        try:
            dt = datetime.strptime(digits[:14], "%Y%m%d%H%M%S")
            return dt.strftime("%Y-%m-%d %H:%M:%S")
        except:
            pass
    elif len(digits) >= 8:
        try:
            dt = datetime.strptime(digits[:8], "%Y%m%d")
            return dt.strftime("%Y-%m-%d 00:00:00")
        except:
            pass
    return None


In [84]:
# ===========================================================
#  工具函数：纠正名称
# ===========================================================
def correct_name(name: str, valid_names: set) -> Dict[str, Union[str, bool]]:
    """纠正识别错误的名称"""
    if name in valid_names:
        return {"name": name, "is_valid": True}

    def edit_distance(s1: str, s2: str) -> int:
        if len(s1) < len(s2):
            return edit_distance(s2, s1)
        if len(s2) == 0:
            return len(s1)

        previous_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row

        return previous_row[-1]

    best_match = min(valid_names, key=lambda x: edit_distance(name, x))
    if edit_distance(name, best_match) <= len(name) // 2:
        return {"name": best_match, "is_valid": True}

    # 如果无法纠正，返回原始名称和无效标识
    print(f"[Warning] 无法纠正名称: {name}")
    return {"name": name, "is_valid": False}

In [85]:
# ===========================================================
#  工具函数：清理名称前后缀
# ===========================================================
def clean_name(name: str, prefix_patterns: List[str], suffix_patterns: List[str]) -> str:
    """
    清理名称中的前缀和后缀。
    :param name: 原始名称
    :param prefix_patterns: 前缀的正则表达式列表
    :param suffix_patterns: 后缀的正则表达式列表
    :return: 清理后的名称
    """
    for pattern in prefix_patterns:
        name = re.sub(pattern, "", name).strip()
    for pattern in suffix_patterns:
        name = re.sub(pattern, "", name).strip()
    return name


In [86]:
def load_json(file_path: str) -> Dict:
    """加载 JSON 文件"""
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def save_json(data: Dict, file_path: str):
    """保存 JSON 文件"""
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def check_info(info1: Dict, info2: Dict) -> bool:
    """检查 info 字段是否一致"""
    keys_to_check = ['game_id', 'uid', 'timezone', 'lang']
    for key in keys_to_check:
        if info1[key] != info2[key]:
            print(f"[Warning] 文件的 info 字段 {key} 不一致：{info1[key]} vs {info2[key]}")
            return False
    return True

def find_overlap(data1: List[Dict], data2: List[Dict]):
    """寻找重合条目，返回重合部分的起始和结束索引"""
    for i in range(len(data1)):
        for j in range(len(data2)):
            if data1[i]['time'] == data2[j]['time'] and \
               data1[i]['pool'] == data2[j]['pool'] and \
               data1[i]['item'] == data2[j]['item']:
                # 检查是否有连续的重合条目
                k = 0
                while i + k < len(data1) and j + k < len(data2) and \
                      data1[i + k]['time'] == data2[j + k]['time'] and \
                      data1[i + k]['pool'] == data2[j + k]['pool'] and \
                      data1[i + k]['item'] == data2[j + k]['item']:
                    k += 1
                if k >= 5:  # 找到连续 5 条以上的重合条目
                    return i, i + k, j, j + k
    return -1, -1, -1, -1

def merge_data(data1: List[Dict], data2: List[Dict], overlap_start1: int, overlap_end1: int, overlap_start2: int, overlap_end2: int) -> List[Dict]:
    """合并数据，保留较长的重合前后部分"""
    # 分割数据
    before_overlap1 = data1[:overlap_start1]
    overlap = data1[overlap_start1:overlap_end1]
    after_overlap1 = data1[overlap_end1:]

    before_overlap2 = data2[:overlap_start2]
    after_overlap2 = data2[overlap_end2:]

    # 比较重合前后的部分长度，保留较长的部分
    before_overlap = before_overlap1 if len(before_overlap1) > len(before_overlap2) else before_overlap2
    after_overlap = after_overlap1 if len(after_overlap1) > len(after_overlap2) else after_overlap2

    # 合并数据
    merged = before_overlap + overlap + after_overlap
    return merged

def merge_json_files(file1: str, file2: str, output_file: str):
    """合并两个 JSON 文件中的 data 部分"""
    json1 = load_json(file1)
    json2 = load_json(file2)

    # 检查 info 字段是否一致
    if not check_info(json1['info'], json2['info']):
        raise ValueError("两个文件的 info 字段不一致，无法合并")

    # 使用时间戳较晚的文件的 info 信息
    info = json2['info'] if json2['info']['export_timestamp'] > json1['info']['export_timestamp'] else json1['info']

    # 寻找重合条目
    start1, end1, start2, end2 = find_overlap(json1['data'], json2['data'])
    if start1 == -1:
        raise ValueError("未找到连续 5 条以上的重合条目，无法合并")

    # 合并 data 部分
    merged_data = merge_data(json1['data'], json2['data'], start1, end1, start2, end2)

    # 构建合并后的 JSON 数据
    merged_json = {
        "info": info,  # 使用时间戳较晚的文件的 info 信息
        "data": merged_data
    }

    # 保存合并后的 JSON 文件
    save_json(merged_json, output_file)


def find_compatible_old_files(new_file_path: Path, existing_files: List[Path]) -> List[Path]:
    """找到与新文件info字段兼容的旧文件列表"""
    compatible_files = []
    
    try:
        # 读取新文件的info信息
        with open(new_file_path, 'r', encoding='utf-8') as f:
            new_data = json.load(f)
        new_info = new_data['info']
        
        for file_path in existing_files:
            try:
                # 读取旧文件的info信息
                with open(file_path, 'r', encoding='utf-8') as f:
                    old_data = json.load(f)
                old_info = old_data['info']
                
                # 检查关键字段是否一致
                if (new_info['game_id'] == old_info['game_id'] and
                    new_info['uid'] == old_info['uid'] and
                    new_info['timezone'] == old_info['timezone'] and
                    new_info['lang'] == old_info['lang']):
                    compatible_files.append(file_path)
                    
            except Exception as e:
                print(f"读取文件 {file_path} 失败: {e}")
                continue
                
    except Exception as e:
        print(f"读取新文件 {new_file_path} 失败: {e}")
    
    return compatible_files

def get_latest_file_by_export_timestamp(files: List[Path]) -> tuple[Optional[Path], int]:
    """根据export_timestamp找到时间戳最晚的文件"""
    if not files:
        return None
    
    latest_file = None
    latest_timestamp = 0
    
    for file_path in files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            timestamp = data['info']['export_timestamp']
            
            if timestamp > latest_timestamp:
                latest_timestamp = timestamp
                latest_file = file_path
                
        except Exception as e:
            print(f"读取文件 {file_path} 失败: {e}")
            continue
    
    return latest_file, latest_timestamp

In [87]:
def run_pipeline(
    image_source: Union[str, List[str]],
    uid: str = "1234567890",
    timezone: int = 8,
    lang: str = "zh-cn",
):
    """主流程：OCR → 解析 → 名称修正 → 导出 → 合并"""
    config = load_config("game_processing_config_ark.json")
    game_data = load_config("game_name_ark.json")
    game_id = config["game_info"]["game_id"]
    game_name = config["game_info"]["game_name"]

    # 构建有效名称集合
    valid_items = set(game_data["character"])  # 角色名称
    valid_pools = set(game_data["pool"])  # 卡池名称

    # 是否启用前后缀清理模块
    enable_clean_name = config["text_processing"]["enable_clean_name"]
    prefix_patterns = config["text_processing"]["patterns"]["prefix_patterns"]
    suffix_patterns = config["text_processing"]["patterns"]["suffix_patterns"]

    if isinstance(image_source, str) and Path(image_source).is_dir():
        image_paths = [str(p) for p in Path(image_source).rglob("*.png")]
    else:
        image_paths = list(image_source)

    # 获取当前时间戳
    export_timestamp = int(datetime.now().timestamp())
    export_time_str = datetime.fromtimestamp(export_timestamp).strftime("%Y%m%d_%H%M%S")

    all_entries = []

    for path in image_paths:
        img = Image.open(path)
        cropped_img = crop_table_area(img, config)
        cropped_img = improve_for_arknights(cropped_img)
        ocr_text = try_ocr(cropped_img)
        entries = parse_ocr_text(ocr_text, config)

        for entry in entries:
            # 清理名称（如果启用）
            if enable_clean_name:
                entry["item"] = clean_name(
                    entry["item"], prefix_patterns, suffix_patterns
                )
                entry["pool"] = clean_name(
                    entry["pool"], prefix_patterns, suffix_patterns
                )

            # 纠正名称
            item_result = correct_name(entry["item"], valid_items)
            pool_result = correct_name(entry["pool"], valid_pools)

            entry["item"] = item_result["name"]
            entry["pool"] = pool_result["name"]
            entry["is_valid"] = item_result["is_valid"] and pool_result["is_valid"]

            # 修复时间
            entry["time"] = fix_timestamp(entry["time"])

        all_entries.extend(entries)

    # 对 data 部分按 time 排序
    all_entries.sort(key=lambda x: x["time"], reverse=True)

    # 构建导出信息
    export_data = {
        "info": {
            "game_id": game_id,
            "game_name": game_name,
            "export_timestamp": export_timestamp,
            "export_app": "ocr_export",
            "export_app_version": "v0.0.1",
            "export_time": datetime.fromtimestamp(export_timestamp).strftime(
                "%Y-%m-%d %H:%M:%S"
            ),
            "uid": uid,
            "timezone": timezone,
            "lang": lang,
            "total_entries": len(all_entries),
        },
        "data": all_entries,
    }
    output_folder = Path("gacha_history")  # 指定输出文件夹
    output_folder.mkdir(exist_ok=True)  # 如果文件夹不存在则创建
    output_file = output_folder / f"{game_id}_{export_time_str}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(export_data, f, ensure_ascii=False, indent=4)
    print(f"已创建新文件: {output_file}")

    # 检查是否存在以 game_id 开头的文件（配置中的game_id）
    existing_files = list(output_folder.glob(f"{game_id}_*.json"))
    # 排除当前刚创建的文件
    existing_files = [f for f in existing_files if f != output_file]
    
    if existing_files:
        print(f"找到 {len(existing_files)} 个历史文件")
        
        # 1. 找到与新文件兼容的旧文件
        compatible_files = find_compatible_old_files(output_file, existing_files)
        print(f"其中 {len(compatible_files)} 个文件与当前账号兼容（相同game_id、uid、timezone、lang）")
        
        if compatible_files:
            # 2. 用时间戳最晚的旧文件与新文件合并
            latest_file, latest_timestamp = get_latest_file_by_export_timestamp(compatible_files)
            
            if latest_file:
                print(f"找到最新兼容文件: {latest_file.name} (导出时间: {datetime.fromtimestamp(latest_timestamp).strftime('%Y-%m-%d %H:%M:%S')})")
                
                try:
                    print(f"尝试合并: {latest_file.name} → {output_file.name}")
                    merge_json_files(str(latest_file), str(output_file), str(output_file))
                except ValueError as e:
                    print(f"合并失败: {e}")
                    print(f"保留两个独立文件: {latest_file.name} 和 {output_file.name}")
                else:
                    print(f"合并完成，结果已保存到 {output_file}")
                    
                    # 删除被合并的旧文件
                    try:
                        latest_file.unlink()
                        print(f"已删除旧文件: {latest_file.name}")
                    except Exception as e:
                        print(f"警告：无法删除旧文件 {latest_file.name}: {e}")
            else:
                print("未找到有效的兼容文件进行合并")
        else:
            print("未找到兼容的历史文件，可能是不同账号的记录，保留独立文件")
    else:
        print(f"未找到历史文件，无需合并，保存文件 {output_file}")

In [88]:

# 示例运行
if __name__ == "__main__":
    image_source = "./images"
    uid = "679034235"
    timezone = 8
    lang = "zh-cn"
    run_pipeline(image_source, uid, timezone, lang)

text: 
以风雪为拆                                  香草                         2025-11-14 11:43:46
以风雪为拆                                    芬                          2025-11-13 11:34:56
以风雪为拆                                  酸糖                         2025-11-12 10:37:07
以风雪为拆                                  验风                         2025-11-11 10:49:29
以风雪为拆                                  古米                         2025-11-10 10:56:13
以风雪为拆                                 月见夜                       2025-11-10 10:56:13
以风雪为拆                                  露托                         2025-11-10 10:56:13
以风雪为拆                                  空爆                         2025-11-10 10:56:13
以风雪为拆                                 调香师                       2025-11-10 10:56:13
以风雪为拆                                  卡强                         2025-11-10 10:56:13


entries: [{'item': '香草', 'pool': '以风雪为拆', 'time': '2025-11-14 11:43:46'}, {'item': '芬', 'pool': '以风雪为拆', 'time': '2025-11-13 11:34:56